In [2]:
import os
import cv2
import sys
import json
import numpy as np
import pandas as pd
from darkflow.net.build import TFNet
print('python version:', sys.version)
print('cv2 version:', cv2.__version__)
## tensorflow-gpu 
## CUDA 7.5

python version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
cv2 version: 3.3.0


In [2]:
# ckeck tensorflow gpu
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# ---
# What returns in cmd
# reference: https://stackoverflow.com/questions/38009682/how-to-tell-if-tensorflow-is-using-gpu-acceleration-from-inside-python-shell
# 2017-11-02 23:23:21.039622: I tensorflow/core/common_runtime/direct_session.cc:300] Device mapping:
# /job:localhost/replica:0/task:0/gpu:0 -> device: 0, name: GeForce 920M, pci bus id: 0000:04:00.0


### Video Frame_caculator Func
This is just for test. Cause it will play in real-time-counting.

In [ ]:
### This is just for test( Could do by mutiple fps with video_lengthSeconds)
# def frame_caculator(video_path):
#     total_frame = 0
#     cap = cv2.VideoCapture(video_path)
#     while(True):
#         try:
#             ret, frame = cap.read()
#             # cv2.imshow('frame',frame)
#             if ret:
#                 total_frame +=1
#                 # Quit
#                 if cv2.waitKey(1) & 0xFF == ord('q'):
#                     break
#             else:
#                 break
#         except:
#             break
#     print('this is end', total_frame)
#     # When everything done, release the capture
#     cap.release()
#     cv2.destroyAllWindows()

# Example frame_caculator
# frame_caculator('./test.avi')

### Video Cut Func

In [ ]:
def video_cut(video_path, video_timeLength, video_fps, startTime_second, endTime_second, 
              output_fps, output_w, output_h, output_name='output.avi'):
    """
        1. Input: video_path
        2. Input: video_timeLength, the [total length of video] in seconds.
        3. Input: video_fps, the [frames per second].
        4. Input: startTime_second, is the [percentage of the spliting_start_time] of the video.
        5. Input: endTime_second, is the [percentage of the spliting_end_time] of the video. 
        6. Input: set toe output video's [Output_fps, output_w, output_h].
        7. Output: will be [.avi] format video with the [split part] of video.
    """
    # Capture video
    cap = cv2.VideoCapture(video_path)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_name, fourcc, output_fps, (output_w, output_h))
    
    # Set specific frame-range of video
    frame_total = video_timeLength * video_fps   # 0 - frame_total
    frame_start = startTime_second * video_fps   
    frame_end = endTime_second * video_fps
    print(frame_start, '--->', frame_end)
    
    # Check time 
    e1 = cv2.getTickCount()
    
    for i in range(frame_start, frame_end):
        
        # Set frame selection with frame place-percent(between 0.0-1.0)
        cap.set(1, i)
        
        # Read and Save
        ret, frame = cap.read()
        print(i)
        if ret:
            
            # Some tricks
            kernel = np.ones((3,3),np.uint8)
            frame = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)
            
            # Resize it 
            frame = cv2.resize(frame,(output_w, output_h), interpolation = cv2.INTER_CUBIC)
            
            # write the frame to the output
            out.write(frame)
            
            # Show 
            cv2.imshow('frame',frame)
            
            # Exit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
            
    # Check time spent
    e2 = cv2.getTickCount()
    print('Cost time:',(e2 - e1)/cv2.getTickFrequency())
    
    # Release everything if job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
# # Video_cut Example 
# video_cut(video_path='./MrSmith.mp4',   # path to access video  
#           video_timeLength=(5*60+8),    # Input video's property: total time-Length in second. 
#           video_fps=30,                 # Input video's property: video's frame per second.
#           startTime_second=(0*60+22),   # cut_start_second:  ex: 20
#           endTime_second=(0*60+40),     # cut_end_second:    ex: 30
#           output_fps=30,                # Output video's property: video's frame per second. 
#           output_h=800,                 # Output video's property: video's height.
#           output_w=600,                 # Output video's property: video's width. 
#           output_name='smith_18.avi')      # Output video's property: video's name. (format can't be changed here.)

### Video Merge

In [ ]:
def videos_merge(video_path_list, output_fps, output_w, output_h, output_name = 'mergeee.avi'):
    """
        1. Input: video_path_list
        2. Input: set toe output video's [Output_fps, output_w, output_h].
        3. Output: will be [.avi] format [merged] video.
    """

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_name, fourcc, output_fps, (output_w, output_h))

    # Check time spent
    e1 = cv2.getTickCount()
    for video in video_path_list:
        print('Start:', video)
        
        # Capture video
        cap = cv2.VideoCapture(video)
        while(True):
            # Capture frame-by-frame
            ret, frame = cap.read() 
            if ret:
                # write the frame to the output
                out.write(frame)

                # Show 
                cv2.imshow('frame',frame)

                # Exit
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break 
        print('End:', video)
        
    # Check time spent
    e2 = cv2.getTickCount()
    print('Cost time:',(e2 - e1)/cv2.getTickFrequency())
    
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()       

In [ ]:
# # videos_merge Example
# video_path_list = ['./catdog_18.avi', './smith_18.avi', './conan_18.avi', './keny_181.avi', './keny_182.avi']
# videos_merge(video_path_list=video_path_list, output_fps=30, output_h=800, output_w=600, output_name='mergeee.avi')

In [ ]:
def video_format_transfer(video_path, output_format='mp4', output_name='video_formatted', output_fps=30, output_w=800, output_h=600):
    
    # Define the codec and create VideoWriter object
    if output_format == 'mp4':
        fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    elif output_format == 'avi':
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
    else:
        fourcc = 0
    
    if fourcc:
        out = cv2.VideoWriter(output_name, fourcc, output_fps, (output_w, output_h))

        # Check time spent
        e1 = cv2.getTickCount()
        # Capture video
        cap = cv2.VideoCapture(video_path)

        while(True):
            # Capture frame-by-frame
            ret, frame = cap.read() 
            if ret:
                # Resize it 
                frame = cv2.resize(frame,(output_w, output_h), interpolation = cv2.INTER_CUBIC)

                # write the frame to the output
                out.write(frame)

                # Show 
                cv2.imshow('frame',frame)

                # Exit
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            else:
                break 

        # Check time spent
        e2 = cv2.getTickCount()
        print('End:', video_path)
        print('Cost time:',(e2 - e1)/cv2.getTickFrequency())

        # When everything done, release the capture
        cap.release()
        cv2.destroyAllWindows()       
    else:
        print('please Input the correct format: ["MP4" or "AVI"] ')

In [ ]:
# video_format_transfer Example :
video_format_transfer('./test.mpg', output_format='avi', output_name='video_formatted.mp4', output_fps=30, output_w=800, output_h=600)

### Problems

In [ ]:
# Tensorflow with gpu then [ "gpu": 0.7 will be working. ]
# Reference: https://www.google.com.tw/search?q=darkflow+gpu&oq=darkflow+gpu&aqs=chrome..69i57.3855j0j7&sourceid=chrome&ie=UTF-8

# >>> conda unistall tensorflow: 1.3.0-py35_0 conda-forge
# >>> conda install tensorflow-gpu or pip install tensorflow-gpu

In [ ]:
# CUDA with opencv? 
# https://jamesbowley.co.uk/buildcompile-opencv-v3-3-on-windows-with-cuda-8-0-and-intel-mkltbb/ 看不懂！？
# https://www.scivision.co/anaconda-python-opencv3/ 

# print(cv2.getBuildInformation())

In [ ]:
# conda config --add channels conda-forge
# conda install --use-local opencv
# sudo apt-get install libavcodec-dev libavformat-dev libavdevice-dev
# sudo apt-get install pkg-config
# sudo apt-get install libgtk2.0-dev



# problem issue # opencv Unable to stop the stream: Inappropriate ioctl for device
# https://stackoverflow.com/questions/42562876/opencv3-error-unable-to-stop-the-stream-inappropriate-ioctl-for-device
# https://stackoverflow.com/questions/41200201/opencv-unable-to-stop-the-stream-inappropriate-ioctl-for-device


# OR JUST INSTALL cv2 version: 3.3.0
# 3.3 problem # scn == 3 || scn == 4 in function cvtColor
# https://github.com/HackerHouseYT/AI-Smart-Mirror/issues/36

#####################################################################################################################

### TFNet Setting

### Video Object Detection with Darkflow

In [ ]:
def VOD_darkflow(video_path, video_fps, video_h, video_w, tf_threshold=0.3, tf_gpu=0.9, output_name='VOD_result.avi'):
    
    # Set TFNet initiail options
    options = {"model": "cfg/yolo.cfg",
               "load": "bin/yolo.weights",
               "threshold": tf_threshold,
               "gpu": tf_gpu}    
    tfnet = TFNet(options)
    
    # Capture Video
    cap = cv2.VideoCapture(video_path)
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_name, fourcc, video_fps, (video_h, video_w))
    
    # Record use
    total_time = 0            # Caculate total time-cost
    tag_list = []             # Store detected object tags
    itemImage_list = []       # Store detected images
    i = 0                     # Skip frame use 
    
    while(True):
        
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Check time 
        e1 = cv2.getTickCount()

#         if ret & ((i%5) == 0):
        if ret:
            # Prediction with darkflow
            print('=== Start Predicting with darkflow ===')
            predictions = tfnet.return_predict(frame)
            for x in range(len(predictions)):
                            if predictions!=[]:
                                topleftx = predictions[x]['topleft']['x']
                                toplefty = predictions[x]['topleft']['y']
                                bottomrightx = predictions[x]['bottomright']['x']
                                bottomrighty = predictions[x]['bottomright']['y']
                                detect = predictions[x]['label']
                                print(predictions[x]['confidence'])
                                if detect not in tag_list:
                                    tag_list.append(detect)
                                    detected_obj = frame[toplefty:bottomrighty, topleftx:bottomrightx] #[height1:height2, width1:width2]
                                    if predictions[x]['confidence'] >= 0.7:
                                        cv2.imwrite('./objImg_high/{0}.png'.format(detect),detected_obj)
                                    elif predictions[x]['confidence'] >= 0.4:
                                        cv2.imwrite('./objImg_mid/{0}.png'.format(detect),detected_obj)
                                    else:
                                        cv2.imwrite('./objImg_low/{0}.png'.format(detect),detected_obj)
                                confidence = str(predictions[x]['confidence'])
                                font = cv2.FONT_HERSHEY_SIMPLEX
                                cv2.rectangle(frame,(topleftx,toplefty),(bottomrightx,bottomrighty),(0,255,0),3)
                                cv2.putText(frame,detect,(bottomrightx-150,bottomrighty-60), font, 2,(0,0,255),2,cv2.LINE_AA)
                                cv2.putText(frame,confidence,(bottomrightx-150,bottomrighty-20), font, 1,(80,50,255),2,cv2.LINE_AA)
        else:
            break
            
        # Display the resulting frame
        cv2.imshow('frame',frame)

        # write the frame to the output
#         out.write(frame)

        # Check time spent
        e2 = cv2.getTickCount()
        print('Cost time:',(e2 - e1)/cv2.getTickFrequency())
        total_time += ((e2 - e1)/cv2.getTickFrequency())
        
        # Skip frame 
        i += 1

        # Quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Print Finished Hint Message
    print("=== VOD_darkflow has finished. ===")
    print("Video was saved as:", output_name)
    
    # Print Total Time Spent
    print("=== Total Time Spent ===")
    print(total_time)
            
    # Print Tag List
    print("=== Object Detected ===")
    print(tag_list)
    
    
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# VOD_darkflow Example:
VOD_darkflow(video_path=0, 
             video_fps=30, 
             video_h=800, 
             video_w=600, 
             output_name='test.avi')

### Demo Use

In [3]:
# Set TFNet (i%500) == 0initiail options
options = {"model": "cfg/yolo.cfg",
           "load": "bin/yolo.weights",
           "threshold": 0.3,
           "gpu": 0.9}

tfnet = TFNet(options)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 269862452 bytes
Finished in 0.131866455078125s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 128)
 Load  |  Yep!  

In [9]:
# 'a' in list(t_dic.keys())
t_dic['a'] = []
t_dic['a'].append('6')
t_dic

{'a': ['6'], 'b': 6}

In [3]:
# https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html

# CAM
# cap = cv2.VideoCapture(0)

# Demo Video
cap = cv2.VideoCapture('./mergeee.avi')

# # Define the codec and create VideoWriter object
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(output_name, fourcc, output_fps, (output_h, output_w))

total_time = 0            # Caculate total time-cost
total_tag_list = []       # Store detected object tags
total_info = {}
high_tag_dict = {}
mid_tag_dict = {}
low_tag_dict = {}
i = 0                     # Skip frame use 

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check time 
    e1 = cv2.getTickCount()

    if ret & (i%5) == 0:
        # Initial dict for each frame
        total_info[i] = []
        
        # Prediction with tfnet 
        print('=== Start Predicting with darkflow ===')
        predictions = tfnet.return_predict(frame)
        if predictions:
            for item in predictions:
                # Prediction information 
                top_left_x = item['topleft']['x']
                top_left_y = item['topleft']['y']
                bot_right_x = item['bottomright']['x']
                bot_right_y = item['bottomright']['y']
                label = item['label']
                confidence = item['confidence']
                
                # Create rectangle for detected object
                cv2.rectangle(frame, (top_left_x, top_left_y), (bot_right_x, bot_right_y), (0,255,0), 3)
                cv2.putText(frame, label, (bot_right_x-150, bot_right_y-60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2, cv2.LINE_AA)
                cv2.putText(frame, str(round(confidence, 3)),(bot_right_x-150,bot_right_y-20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1,(80,50,255),2,cv2.LINE_AA)

                # Detected object image 
                detected_obj = frame[top_left_y:bot_right_y, top_left_x:bot_right_x] #[height1:height2, width1:width2]
                
                # Store information and images
                if label not in total_tag_list:
                    total_tag_list.append(label)
                
                total_info[i].append((label, str(round(confidence, 3))))
                                     
                if confidence > 0.7:
                    if label not in list(high_tag_dict.keys()):
                        high_tag_dict[label] = (str(round(confidence, 2))+'_'+str(i))
                        if not os.path.exists('./objImg_high'):
                            os.makedirs('./objImg_high')
                        cv2.imwrite('./objImg_high/{0}_{1}.png'.format(label, (str(round(confidence, 2))+'_'+str(i))), detected_obj)
                            
                elif confidence > 0.4:
                    if label not in list(mid_tag_dict.keys()):
                        mid_tag_dict[label] = (str(round(confidence, 2))+'_'+str(i))
                        if not os.path.exists('./objImg_mid'):
                            os.makedirs('./objImg_mid')
                        cv2.imwrite('./objImg_mid/{0}_{1}.png'.format(label, (str(round(confidence, 2))+'_'+str(i))), detected_obj)
 
                else:
                    if label not in list(low_tag_dict.keys()):
                        low_tag_dict[label] = (str(round(confidence, 2))+'_'+str(i))
                        if not os.path.exists('./objImg_low'):
                            os.makedirs('./objImg_low')
                        cv2.imwrite('./objImg_low/{0}_{1}.png'.format(label, (str(round(confidence, 2))+'_'+str(i))), detected_obj)
        
    # Display the resulting frame
#     cv2.imshow('frame',frame)
    
    # write the frame to the output
#     out.write(frame)
    
    # Check time spent
    e2 = cv2.getTickCount()
    print('Cost time:',(e2 - e1)/cv2.getTickFrequency())
    
    # Skip frame 
    i += 1
    
    # Quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Print Tag List
print("=== Object Detected ===")

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

=== Start Predicting with darkflow ===
Cost time: 8.38959978
Cost time: 6.64e-06
=== Start Predicting with darkflow ===
Cost time: 0.567260936
Cost time: 5.008e-06
=== Start Predicting with darkflow ===
Cost time: 0.562244104
=== Start Predicting with darkflow ===
Cost time: 0.546485416
Cost time: 5.861e-06
=== Start Predicting with darkflow ===
Cost time: 0.581933836
Cost time: 5.299e-06
=== Start Predicting with darkflow ===
Cost time: 0.524800843
=== Start Predicting with darkflow ===
Cost time: 0.566404281
Cost time: 4.881e-06
=== Start Predicting with darkflow ===
Cost time: 0.565133061
Cost time: 3.754e-06
=== Start Predicting with darkflow ===
Cost time: 0.592110048
=== Start Predicting with darkflow ===
Cost time: 0.633092236
Cost time: 6.226e-06
=== Start Predicting with darkflow ===
Cost time: 0.564038886
Cost time: 5.601e-06
=== Start Predicting with darkflow ===
Cost time: 0.571781619
=== Start Predicting with darkflow ===
Cost time: 0.604738716
Cost time: 9.643e-06
=== Sta

KeyboardInterrupt: 

In [17]:
# total_tag_list
# total_info
# high_tag_dict
# mid_tag_dict
# low_tag_dict
# low_tag_dict.items()
# low_tag_dict.keys()
# pd.DataFrame(low_tag_dict.items(), columns=[low_tag_dict.keys])
# pd.DataFrame(d.items(), columns=['Date', 'DateValue'])



import csv
import json

def to_json(dicto, json_file_name='test.json'):
    js = json.dumps(dicto)
    # Open new json file if not exist it will create
    fp = open('test.json', 'a')
    # write to json file
    fp.write(js)
    # close the connection
    fp.close()
    
to_json(dicto=low_tag_dict, json_file_name='low_tag_dict.json')

In [1]:
import os
import cv2
import sys
import json
import numpy as np
import pandas as pd
from darkflow.net.build import TFNet
print('python version:', sys.version)
print('cv2 version:', cv2.__version__)
## tensorflow-gpu 
## CUDA 7.5

options = {"model": "cfg/yolo.cfg",
           "load": "bin/yolo.weights",
           "threshold": 0.3,
           "gpu": 0.9}

tfnet = TFNet(options)

python version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
cv2 version: 3.3.0
Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 269862452 bytes
Finished in 0.07049036026000977s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep! 